# Data Preparation


## 0. Import Libraries


In [41]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import numpy as np

## 1. Read the data

process

- merge data from 57-68
- clean data


In [ ]:
def preprocess_dataframe(df):
    # Strip column names
    df.columns = df.columns.str.strip()
    print("Column names after stripping:", df.columns.tolist())

    # Delete missing values from columns
    df = df.dropna(subset=["เวลา"], axis=0)

    return df

# Calculate duration in hours with better error handling
def calculate_duration(time_str):
    if pd.isna(time_str):
        return None, None, None

    # Clean the string: remove 'น', dots, and extra spaces
    time_str = str(time_str).replace("น.", "").replace("น", "").strip()
    times = time_str.split("-")

    if len(times) == 2:
        try:
            # Remove trailing dots and spaces before converting
            start_str = times[0].strip().rstrip(".")
            end_str = times[1].strip().rstrip(".")

            start = float(start_str)  # Convert 18.00 to 18.0
            end = float(end_str)  # Convert 22.00 to 22.0

            # Handle overnight times
            if end < start:
                end += 24
            duration = np.ceil(end - start)
            return duration, start, end
        except (ValueError, ZeroDivisionError):
            return None, None, None

    return None, None, None


# Simplified event classification
def classify_event_simple(start_time, end_time):
    if pd.isna(start_time) or pd.isna(end_time):
        return None

    # Morning: 09.00-12.59
    if 8.0 <= start_time <= 13.0 and 8.30 <= end_time <= 13.0:
        return "Morning"
    # Afternoon: 13.00-17.59
    elif 13.0 <= start_time <= 18.0 and 13.0 <= end_time <= 18.0:
        return "Afternoon"
    # Night: 18.00-24.00
    elif 18.0 <= start_time <= 24.0 and 18.0 <= end_time <= 24.0:
        return "Night"
    else:
        return "All Day"


month_map = {
    "ม.ค": 1,
    "มค": 1,
    "มกราคม": 1,
    "ก.พ": 2,
    "กพ": 2,
    "กุมภาพันธ์": 2,
    "มี.ค": 3,
    "มีค": 3,
    "มีนาคม": 3,
    "เม.ย": 4,
    "เมย": 4,
    "เมษายน": 4,
    "พ.ค": 5,
    "พค": 5,
    "พฤษภาคม": 5,
    "มิ.ย": 6,
    "มิย": 6,
    "มิถุนายน": 6,
    "ก.ค": 7,
    "กค": 7,
    "กรกฎาคม": 7,
    "ส.ค": 8,
    "สค": 8,
    "สิงหาคม": 8,
    "ก.ย": 9,
    "กย": 9,
    "กันยายน": 9,
    "ต.ค": 10,
    "ตค": 10,
    "ตุลาคม": 10,
    "พ.ย": 11,
    "พย": 11,
    "พฤศจิกายน": 11,
    "ธ.ค": 12,
    "ธค": 12,
    "ธันวาคม": 12,
}


def thai_to_datetime(date_str):
    """แปลงวันที่ภาษาไทย (พ.ศ.) เป็น pandas.Timestamp"""
    if pd.isna(date_str):
        return pd.NaT
    # ลบจุดและเว้นวรรคแล้ว strip
    s = re.sub(r"[\. ]+", "", str(date_str).strip())
    # แยกวัน เดือน ปี
    m = re.match(r"(\d{1,2})([^\d]+)(\d{2,4})", s)
    if not m:
        return pd.NaT
    day, mon_str, year_str = m.groups()
    # แปลงเป็น int
    day = int(day)
    mon = month_map.get(mon_str)
    if mon is None:
        return pd.NaT
    year_be = int(year_str)
    if year_be < 100:
        year_be += 2500
    year_ce = year_be - 543
    return pd.Timestamp(year=year_ce, month=mon, day=day)


room_cols = [
    "ห้องประชุม",
    "อยุธยา - อาเซียน",
    "อยุธยา-ฮอลันดา",
    "อยุธยา - โปรตุเกส",
    "ห้องประชุม 317",
    "หอประชุม",
]
# Create column 'room'
def extract_room(row, room_cols=room_cols):
    for col in room_cols:
        val = row.get(col)
        if pd.notna(val) and str(val).strip() != "":
            return val
    return None


# Drop unnecessary columns
def drop_unnecessary_columns(df, columns_to_drop=None):
    if columns_to_drop is None:
        # Default columns to drop if not provided
        columns_to_drop = [
            "เลขที่",
            "วันที่รับ",
            "ชื่อ-สกุลผู้ขอใช้",
            "ใช้ในวันที่",
            "เวลา",
            "ห้องประชุม",
            "อยุธยา - อาเซียน",
            "อยุธยา-ฮอลันดา",
            "อยุธยา - โปรตุเกส",
            "ห้องประชุม 317",
            "หอประชุม",
            "เรื่อง",
            "Unnamed: 13",
        ]
    return df.drop(columns=columns_to_drop, errors="ignore")


def normalize_room(name):
    if pd.isna(name):
        return None

    # ทำความสะอาดสตริง: ลบจุด, ยุบช่องว่าง, lowercase
    s = re.sub(r'[^\wก-๙]', '', str(name)).lower()

    # จับคู่ “ห้องประชุม สนอ”
    if 'ห้องประชุมสนอ' in s:
        return 'ห้องประชุม สนอ.'

    # หอประชุม
    if 'หอประชุม' in s:
        return 'หอประชุม'

    # อยุธยา-อาเซียน
    if 'อยุธยา' in s and 'อาเซียน' in s:
        return 'อยุธยา-อาเซียน'

    # อยุธยา-ฮอลันดา
    if 'อยุธยา' in s and 'ฮอลันดา' in s:
        return 'อยุธยา-ฮอลันดา'

    # อยุธยา-โปรตุเกส
    if 'อยุธยา' in s and 'โปรตุเกส' in s:
        return 'อยุธยา-โปรตุเกส'

    # แยกห้อง 317
    if '317' in s:
        has_on = 'บน' in s
        has_under = 'ล่าง' in s
        if has_on and has_under:
            return 'ห้องประชุม 317'
        if has_on:
            return 'ห้องประชุม 317 บน'
        if has_under:
            return 'ห้องประชุม 317 ล่าง'
        return 'ห้องประชุม 317'

    # ห้องประชุม 1 และ 2
    if re.search(r'ห้องประชุม1', s):
        return 'ห้องประชุม 1'
    if 'ห้องประชุม2' in s:
        return 'ห้องประชุม 2'

    # ตึก 70 ล้าน
    if '70ล้าน' in s:
        return 'ห้องประชุมบ้านพลูหลวง'

    # บ้านพลูหลวง
    if 'พลูหลวง' in s:
        return 'ห้องประชุมบ้านพลูหลวง'

    # โรงอาหารเก่า
    if 'โรงอาหารเก่า' in s:
        return 'โรงยิม'

    # สนามหญ้าอาคาร 2
    if 'สนามหญ้าอาคาร2' in s:
        return 'สนามฟุตบอล'

    return None

In [43]:
df_57 = pd.read_excel("../data/ห้องประชุม ไปราชการปี 2557.xlsx", header=0, skiprows=1)
print("DataFrame shape:", df_57.shape)

df_57 = preprocess_dataframe(df_57)
# Apply the function
df_57["duration_hours"], df_57["start_time"], df_57["end_time"] = zip(
    *df_57["เวลา"].apply(calculate_duration)
)

df_57["event_period"] = df_57.apply(
    lambda row: classify_event_simple(row["start_time"], row["end_time"]), axis=1
)

df_57["ใช้ในวันที่ (datetime)"] = df_57["ใช้ในวันที่"].apply(thai_to_datetime)

room_cols = [
    "ห้องประชุม",
    "อยุธยา - อาเซียน",
    "อยุธยา-ฮอลันดา",
    "อยุธยา - โปรตุเกส",
    "ห้องประชุม 317",
    "หอประชุม",
]
df_57["room"] = df_57.apply(lambda row: extract_room(row, room_cols), axis=1)

df_57 = drop_unnecessary_columns(df_57)

# Save the cleaned DataFrame to a new Excel file
df_57.to_excel("../data/clean/cleaned_ห้องประชุม_2557.xlsx", index=False)

df_57.head()

DataFrame shape: (638, 14)
Column names after stripping: ['เลขที่', 'วันที่รับ', 'ชื่อ-สกุลผู้ขอใช้', 'ใช้ในวันที่', 'เวลา', 'ห้องประชุม', 'อยุธยา - อาเซียน', 'อยุธยา-ฮอลันดา', 'อยุธยา - โปรตุเกส', 'ห้องประชุม 317', 'หอประชุม', 'เรื่อง', 'หน่วยงานที่ขอใช้', 'Unnamed: 13']


,หน่วยงานที่ขอใช้,duration_hours,start_time,end_time,event_period,ใช้ในวันที่ (datetime),room
0,รพ.อยุธยา,4.0,18.0,22.0,Night,2013-09-27,หอประชุม
1,กบค,4.0,9.0,12.3,Morning,2013-10-29,อยุธยา-อาเซียน
3,กนผ.,NaN,NaN,NaN,None,2013-10-16,อยุธยา-อาเซียน
4,กบค,2.0,13.3,15.3,Afternoon,2013-10-09,อยุธยา - ฮอลันดา
5,กบศ,3.0,9.3,12.3,Morning,2013-10-10,อยุธยา - ฮอลันดา


In [44]:
df_58 = pd.read_excel("../data/ห้องประชุม ไปราชการปี 2558.xlsx", header=0, skiprows=1)
print("DataFrame shape:", df_58.shape)

df_58 = preprocess_dataframe(df_58)
# Apply the function
df_58["duration_hours"], df_58["start_time"], df_58["end_time"] = zip(
    *df_58["เวลา"].apply(calculate_duration)
)

df_58["event_period"] = df_58.apply(
    lambda row: classify_event_simple(row["start_time"], row["end_time"]), axis=1
)

df_58["ใช้ในวันที่ (datetime)"] = df_58["ใช้ในวันที่"].apply(thai_to_datetime)

room_cols = [
    "ห้องประชุม  1/2",
    "อยุธยา - อาเซียน",
    "อยุธยา-ฮอลันดา",
    "อยุธยา - โปรตุเกส",
    "ห้องประชุม 317",
    "หอประชุม",
]
df_58["room"] = df_58.apply(lambda row: extract_room(row, room_cols), axis=1)

columns_to_drop = [
    "เลขที่",
    "วันที่รับ",
    "ชื่อ-สกุลผู้ขอใช้",
    "ใช้ในวันที่",
    "เวลา",
    "ห้องประชุม  1/2",
    "อยุธยา - อาเซียน",
    "อยุธยา-ฮอลันดา",
    "อยุธยา - โปรตุเกส",
    "ห้องประชุม 317",
    "หอประชุม",
    "เรื่อง",
]
df_58 = drop_unnecessary_columns(df_58, columns_to_drop)

# Save the cleaned DataFrame to a new Excel file
df_58.to_excel("../data/clean/cleaned_ห้องประชุม_2558.xlsx", index=False)

df_58.head()

DataFrame shape: (683, 13)
Column names after stripping: ['เลขที่', 'วันที่รับ', 'ชื่อ-สกุลผู้ขอใช้', 'ใช้ในวันที่', 'เวลา', 'ห้องประชุม  1/2', 'อยุธยา - อาเซียน', 'อยุธยา-ฮอลันดา', 'อยุธยา - โปรตุเกส', 'ห้องประชุม 317', 'หอประชุม', 'เรื่อง', 'หน่วยงานที่ขอใช้']


,หน่วยงานที่ขอใช้,duration_hours,start_time,end_time,event_period,ใช้ในวันที่ (datetime),room
1,สวพ,2.0,10.0,12.0,Morning,2014-10-03,อยุธยา - ฮอลันดา
2,กบค,3.0,10.0,13.0,Morning,2014-10-16,อยุธยา - อาเซียน
6,คคศ,8.0,8.3,16.3,All Day,NaT,หอประชุม
7,คคศ,8.0,8.3,16.3,All Day,2014-12-17,หอประชุม
8,คคศ,8.0,8.3,16.3,All Day,2015-01-28,หอประชุม


In [45]:
df_58 = pd.read_excel("../data/ห้องประชุม ไปราชการปี 2558.xlsx", header=0, skiprows=1)
print("DataFrame shape:", df_58.shape)

df_58 = preprocess_dataframe(df_58)
# Apply the function
df_58["duration_hours"], df_58["start_time"], df_58["end_time"] = zip(
    *df_58["เวลา"].apply(calculate_duration)
)

df_58["event_period"] = df_58.apply(
    lambda row: classify_event_simple(row["start_time"], row["end_time"]), axis=1
)

df_58["ใช้ในวันที่ (datetime)"] = df_58["ใช้ในวันที่"].apply(thai_to_datetime)

room_cols = [
    "ห้องประชุม  1/2",
    "อยุธยา - อาเซียน",
    "อยุธยา-ฮอลันดา",
    "อยุธยา - โปรตุเกส",
    "ห้องประชุม 317",
    "หอประชุม",
]
df_58["room"] = df_58.apply(lambda row: extract_room(row, room_cols), axis=1)

columns_to_drop = [
    "เลขที่",
    "วันที่รับ",
    "ชื่อ-สกุลผู้ขอใช้",
    "ใช้ในวันที่",
    "เวลา",
    "ห้องประชุม  1/2",
    "อยุธยา - อาเซียน",
    "อยุธยา-ฮอลันดา",
    "อยุธยา - โปรตุเกส",
    "ห้องประชุม 317",
    "หอประชุม",
    "เรื่อง",
]
df_58 = drop_unnecessary_columns(df_58, columns_to_drop)

# Save the cleaned DataFrame to a new Excel file
df_58.to_excel("../data/clean/cleaned_ห้องประชุม_2558.xlsx", index=False)

df_58.head()

DataFrame shape: (683, 13)
Column names after stripping: ['เลขที่', 'วันที่รับ', 'ชื่อ-สกุลผู้ขอใช้', 'ใช้ในวันที่', 'เวลา', 'ห้องประชุม  1/2', 'อยุธยา - อาเซียน', 'อยุธยา-ฮอลันดา', 'อยุธยา - โปรตุเกส', 'ห้องประชุม 317', 'หอประชุม', 'เรื่อง', 'หน่วยงานที่ขอใช้']


,หน่วยงานที่ขอใช้,duration_hours,start_time,end_time,event_period,ใช้ในวันที่ (datetime),room
1,สวพ,2.0,10.0,12.0,Morning,2014-10-03,อยุธยา - ฮอลันดา
2,กบค,3.0,10.0,13.0,Morning,2014-10-16,อยุธยา - อาเซียน
6,คคศ,8.0,8.3,16.3,All Day,NaT,หอประชุม
7,คคศ,8.0,8.3,16.3,All Day,2014-12-17,หอประชุม
8,คคศ,8.0,8.3,16.3,All Day,2015-01-28,หอประชุม


In [46]:
df_59 = pd.read_excel("../data/ห้องประชุม ไปราชการปี 2559.xlsx", header=0, skiprows=1)
print("DataFrame shape:", df_59.shape)

df_59 = preprocess_dataframe(df_59)
# Apply the function
df_59["duration_hours"], df_59["start_time"], df_59["end_time"] = zip(
    *df_59["เวลา"].apply(calculate_duration)
)

df_59["event_period"] = df_59.apply(
    lambda row: classify_event_simple(row["start_time"], row["end_time"]), axis=1
)

df_59["ใช้ในวันที่ (datetime)"] = df_59["ใช้ในวันที่"].apply(thai_to_datetime)

room_cols = [
    'ห้องประชุม  1/2', 'อยุธยา - อาเซียน',
       'อยุธยา-ฮอลันดา', 'อยุธยา - โปรตุเกส', 'ห้องประชุม 317', 'หอประชุม',
]
df_59["room"] = df_59.apply(lambda row: extract_room(row, room_cols), axis=1)

columns_to_drop = [
    'เลขที่', 'วันที่', 'ชื่อ-สกุลผู้ขอใช้',
       'ใช้ในวันที่', 'เวลา', 'ห้องประชุม  1/2', 'อยุธยา - อาเซียน',
       'อยุธยา-ฮอลันดา', 'อยุธยา - โปรตุเกส', 'ห้องประชุม 317', 'หอประชุม',
       'เรื่อง',
]
df_59 = drop_unnecessary_columns(df_59, columns_to_drop)

# Save the cleaned DataFrame to a new Excel file
df_59.to_excel("../data/clean/cleaned_ห้องประชุม_2559.xlsx", index=False)

df_59.head()

DataFrame shape: (845, 13)
Column names after stripping: ['เลขที่', 'หน่วยงานที่ขอใช้', 'วันที่', 'ชื่อ-สกุลผู้ขอใช้', 'ใช้ในวันที่', 'เวลา', 'ห้องประชุม  1/2', 'อยุธยา - อาเซียน', 'อยุธยา-ฮอลันดา', 'อยุธยา - โปรตุเกส', 'ห้องประชุม 317', 'หอประชุม', 'เรื่อง']


,หน่วยงานที่ขอใช้,duration_hours,start_time,end_time,event_period,ใช้ในวันที่ (datetime),room
1,งานประกัน,2.0,15.3,17.0,Afternoon,2015-10-05,อยุธยา - อาเซียน
2,สภา,1.0,15.3,16.3,Afternoon,2015-10-07,อยุธยา - โปรตุเกส
3,สภา,1.0,15.3,16.3,Afternoon,2015-10-09,อยุธยา - โปรตุเกส
4,ควจ,1.0,11.0,12.0,Morning,2015-10-12,ห้องประชุม 317
6,สวพ,1.0,11.0,12.0,Morning,2015-10-06,อยุธยา - โปรตุเกส


In [47]:
df_60_1 = pd.read_excel("../data/ห้องประชุม ไปราชการปี 2560.xlsx", header=0, skiprows=1, sheet_name=0)
print("DataFrame shape:", df_60_1.shape)

df_60_1 = preprocess_dataframe(df_60_1)
# Apply the function
df_60_1["duration_hours"], df_60_1["start_time"], df_60_1["end_time"] = zip(
    *df_60_1["เวลา"].apply(calculate_duration)
)

df_60_1["event_period"] = df_60_1.apply(
    lambda row: classify_event_simple(row["start_time"], row["end_time"]), axis=1
)

df_60_1["ใช้ในวันที่ (datetime)"] = df_60_1["ใช้ในวันที่"].apply(thai_to_datetime)

room_cols = [
'ห้องประชุม สนอ.', 'ห้องประชุมตึก 70 ล้าน',
       'ห้องประชุม  1/2', 'อยุธยา - อาเซียน', 'อยุธยา-ฮอลันดา',
       'อยุธยา - โปรตุเกส', 'ห้องประชุม 317', 'หอประชุม'
]
df_60_1["room"] = df_60_1.apply(lambda row: extract_room(row, room_cols), axis=1)

columns_to_drop = [
'เลขที่', 'วันที่', 'ชื่อ-สกุลผู้ขอใช้',
       'ใช้ในวันที่', 'เวลา', 'ห้องประชุม สนอ.', 'ห้องประชุมตึก 70 ล้าน',
       'ห้องประชุม  1/2', 'อยุธยา - อาเซียน', 'อยุธยา-ฮอลันดา',
       'อยุธยา - โปรตุเกส', 'ห้องประชุม 317', 'หอประชุม', 'เรื่อง',
]
df_60_1 = drop_unnecessary_columns(df_60_1, columns_to_drop)

# Save the cleaned DataFrame to a new Excel file
df_60_1.to_excel("../data/clean/cleaned_ห้องประชุม_2560_1.xlsx", index=False)

df_60_1.head()

DataFrame shape: (1025, 15)
Column names after stripping: ['เลขที่', 'หน่วยงานที่ขอใช้', 'วันที่', 'ชื่อ-สกุลผู้ขอใช้', 'ใช้ในวันที่', 'เวลา', 'ห้องประชุม สนอ.', 'ห้องประชุมตึก 70 ล้าน', 'ห้องประชุม  1/2', 'อยุธยา - อาเซียน', 'อยุธยา-ฮอลันดา', 'อยุธยา - โปรตุเกส', 'ห้องประชุม 317', 'หอประชุม', 'เรื่อง']


,หน่วยงานที่ขอใช้,duration_hours,start_time,end_time,event_period,ใช้ในวันที่ (datetime),room
1,กนผ,3.0,14.0,16.3,Afternoon,2016-01-05,อยุธยา - โปรตุเกส
2,กบค,3.0,9.3,12.3,Morning,2017-01-31,อยุธยา - อาเซียน
4,อาคาร,4.0,8.3,12.0,Morning,2017-01-06,ห้องประชุม 1
5,กบค,7.0,9.0,15.3,All Day,2017-01-10,อยุธยา - โปรตุเกส
7,ควจ,3.0,13.0,16.0,Afternoon,2017-01-11,ห้องประชุม 317


In [48]:
df_60_2 = pd.read_excel("../data/ห้องประชุม ไปราชการปี 2560.xlsx", header=0, skiprows=1, sheet_name=1)
print("DataFrame shape:", df_60_2.shape)

df_60_2 = preprocess_dataframe(df_60_2)
# Apply the function
df_60_2["duration_hours"], df_60_2["start_time"], df_60_2["end_time"] = zip(
    *df_60_2["เวลา"].apply(calculate_duration)
)

df_60_2["event_period"] = df_60_2.apply(
    lambda row: classify_event_simple(row["start_time"], row["end_time"]), axis=1
)

df_60_2["ใช้ในวันที่ (datetime)"] = df_60_2["ใช้ในวันที่"].apply(thai_to_datetime)

room_cols = [
    'ห้องประชุม  1/2', 'อยุธยา - อาเซียน', 'อยุธยา-ฮอลันดา',
    'อยุธยา - โปรตุเกส', 'ห้องประชุม 317', 'หอประชุม'
]
df_60_2["room"] = df_60_2.apply(lambda row: extract_room(row, room_cols), axis=1)

columns_to_drop = [
    'เลขที่', 'ชื่อ-สกุลผู้ขอใช้', 'ใช้ในวันที่', 'เวลา',
       'ห้องประชุม  1/2', 'อยุธยา - อาเซียน', 'อยุธยา-ฮอลันดา',
       'อยุธยา - โปรตุเกส', 'ห้องประชุม 317', 'หอประชุม', 'เรื่อง',
]
df_60_2 = drop_unnecessary_columns(df_60_2, columns_to_drop)

# Save the cleaned DataFrame to a new Excel file
df_60_2.to_excel("../data/clean/cleaned_ห้องประชุม_2560_2.xlsx", index=False)

df_60_2.head(10)

DataFrame shape: (188, 12)
Column names after stripping: ['เลขที่', 'หน่วยงานที่ขอใช้', 'ชื่อ-สกุลผู้ขอใช้', 'ใช้ในวันที่', 'เวลา', 'ห้องประชุม  1/2', 'อยุธยา - อาเซียน', 'อยุธยา-ฮอลันดา', 'อยุธยา - โปรตุเกส', 'ห้องประชุม 317', 'หอประชุม', 'เรื่อง']


,หน่วยงานที่ขอใช้,duration_hours,start_time,end_time,event_period,ใช้ในวันที่ (datetime),room
1,กนผ,3.0,14.0,16.3,Afternoon,2016-01-05,อยุธยา - โปรตุเกส
2,กบค,3.0,9.3,12.3,Morning,2017-01-31,อยุธยา - อาเซียน
4,อาคาร,4.0,8.3,12.0,Morning,2017-01-06,ห้องประชุม 1
5,กบค,7.0,9.0,15.3,All Day,2017-01-10,อยุธยา - โปรตุเกส
7,ควจ,3.0,13.0,16.0,Afternoon,2017-01-11,ห้องประชุม 317
8,กนผ,3.0,9.0,12.0,Morning,2017-01-13,อยุธยา - อาเซียน
9,ควท,5.0,7.0,12.0,All Day,2017-01-24,หอประชุม
11,กบศ,2.0,14.0,16.0,Afternoon,2017-01-11,อยุธยา - ฮอลันดา
12,กพน,7.0,16.3,23.0,All Day,2017-01-14,หอประชุม
14,งานประกัน,3.0,13.3,16.3,Afternoon,2017-01-12,อยุธยา - โปรตุเกส


In [49]:
df_60_3 = pd.read_excel("../data/ห้องประชุม ไปราชการปี 2560.xlsx", header=0, skiprows=1, sheet_name=2)
print("DataFrame shape:", df_60_3.shape)

df_60_3 = preprocess_dataframe(df_60_3)
# Apply the function
df_60_3["duration_hours"], df_60_3["start_time"], df_60_3["end_time"] = zip(
    *df_60_3["เวลา"].apply(calculate_duration)
)

df_60_3["event_period"] = df_60_3.apply(
    lambda row: classify_event_simple(row["start_time"], row["end_time"]), axis=1
)

df_60_3["ใช้ในวันที่ (datetime)"] = df_60_3["ใช้ในวันที่"].apply(thai_to_datetime)

room_cols = [
        'ห้องประชุม  1/2', 'อยุธยา - อาเซียน', 'อยุธยา-ฮอลันดา',
        'อยุธยา - โปรตุเกส', 'ห้องประชุม 317', 'หอประชุม'
]
df_60_3["room"] = df_60_3.apply(lambda row: extract_room(row, room_cols), axis=1)

columns_to_drop = [
    'เลขที่', 'ชื่อ-สกุลผู้ขอใช้', 'ใช้ในวันที่', 'เวลา',
        'ห้องประชุม  1/2', 'อยุธยา - อาเซียน', 'อยุธยา-ฮอลันดา',
        'อยุธยา - โปรตุเกส', 'ห้องประชุม 317', 'หอประชุม', 'เรื่อง'
]
df_60_3 = drop_unnecessary_columns(df_60_3, columns_to_drop)

# Save the cleaned DataFrame to a new Excel file
df_60_3.to_excel("../data/clean/cleaned_ห้องประชุม_2560_3.xlsx", index=False)

df_60_3.head(10)

DataFrame shape: (160, 12)
Column names after stripping: ['เลขที่', 'หน่วยงานที่ขอใช้', 'ชื่อ-สกุลผู้ขอใช้', 'ใช้ในวันที่', 'เวลา', 'ห้องประชุม  1/2', 'อยุธยา - อาเซียน', 'อยุธยา-ฮอลันดา', 'อยุธยา - โปรตุเกส', 'ห้องประชุม 317', 'หอประชุม', 'เรื่อง']


,หน่วยงานที่ขอใช้,duration_hours,start_time,end_time,event_period,ใช้ในวันที่ (datetime),room
1,สภา,3.0,13.3,16.0,Afternoon,2016-10-03,อยุธยา - ฮอลันดา
2,สภา,5.0,15.0,20.0,All Day,2016-10-18,อยุธยา - โปรตุเกส
3,สภา,4.0,13.0,16.3,Afternoon,2016-10-19,อยุธยา - โปรตุเกส
5,สภา,2.0,13.3,15.0,Afternoon,2016-10-06,อยุธยา - ฮอลันดา
6,สภา,3.0,14.0,16.3,Afternoon,2016-10-07,อยุธยา - โปรตุเกส
8,กนผ,3.0,9.3,12.0,Morning,2016-10-07,อยุธยา - อาเซียน
10,คคศ,4.0,13.0,16.3,Afternoon,2016-10-20,อยุธยา - โปรตุเกส
11,ตสน,3.0,9.3,12.0,Morning,2016-10-21,อยุธยา - อาเซียน
13,กบศ,3.0,13.0,16.0,Afternoon,2016-10-11,อยุธยา - โปรตุเกส
14,คคศ,8.0,8.3,16.3,All Day,NaT,หอประชุม


In [50]:
df_61 = pd.read_excel("../data/ห้องประชุม ไปราชการปี 2561.xlsx", header=0, skiprows=1)
print("DataFrame shape:", df_61.shape)

df_61 = preprocess_dataframe(df_61)
# Apply the function
df_61["duration_hours"], df_61["start_time"], df_61["end_time"] = zip(
    *df_61["เวลา"].apply(calculate_duration)
)

df_61["event_period"] = df_61.apply(
    lambda row: classify_event_simple(row["start_time"], row["end_time"]), axis=1
)

df_61["ใช้ในวันที่ (datetime)"] = df_61["ใช้ในวันที่"].apply(thai_to_datetime)

room_cols = [
        'ห้องประชุม สนอ.', 'ห้องประชุมตึก 70 ล้าน',
        'ห้องประชุม  1/2', 'อยุธยา - อาเซียน', 'อยุธยา - ฮอลันดา',
        'อยุธยา - โปรตุเกส', 'ห้องประชุม 317', 'หอประชุม'
]
df_61["room"] = df_61.apply(lambda row: extract_room(row, room_cols), axis=1)

columns_to_drop = [
    'เลขที่', 'วันที่', 'ชื่อ-สกุลผู้ขอใช้',
       'ใช้ในวันที่', 'เวลา', 'ห้องประชุม สนอ.', 'ห้องประชุมตึก 70 ล้าน',
       'ห้องประชุม  1/2', 'อยุธยา - อาเซียน', 'อยุธยา - ฮอลันดา',
       'อยุธยา - โปรตุเกส', 'ห้องประชุม 317', 'หอประชุม', 'เรื่อง',
]
df_61 = drop_unnecessary_columns(df_61, columns_to_drop)

# Save the cleaned DataFrame to a new Excel file
df_61.to_excel("../data/clean/cleaned_ห้องประชุม_2561.xlsx", index=False)

df_61.head(10)

DataFrame shape: (1157, 15)
Column names after stripping: ['เลขที่', 'หน่วยงานที่ขอใช้', 'วันที่', 'ชื่อ-สกุลผู้ขอใช้', 'ใช้ในวันที่', 'เวลา', 'ห้องประชุม สนอ.', 'ห้องประชุมตึก 70 ล้าน', 'ห้องประชุม  1/2', 'อยุธยา - อาเซียน', 'อยุธยา - ฮอลันดา', 'อยุธยา - โปรตุเกส', 'ห้องประชุม 317', 'หอประชุม', 'เรื่อง']


,หน่วยงานที่ขอใช้,duration_hours,start_time,end_time,event_period,ใช้ในวันที่ (datetime),room
1,กบค,2.0,10.0,12.0,Morning,2018-01-18,อยุธยา - ฮอลันดา
3,สภา,4.0,13.0,16.3,Afternoon,2018-01-11,อยุธยา - โปรตุเกส
4,สภา,8.0,9.0,16.3,All Day,2018-01-26,อยุธยา - โปรตุเกส
5,กบค,3.0,9.3,12.3,Morning,2018-01-30,อยุธยา - อาเซียน
7,ปชส,3.0,13.3,16.0,Afternoon,2018-01-17,อยุธยา - อาเซียน
8,กบค,3.0,9.3,12.0,Morning,2018-01-17,อยุธยา - โปรตุเกส
9,สอศ,9.0,8.0,16.3,All Day,NaT,หอประชุม
10,กบค,2.0,10.0,12.0,Morning,2018-01-22,อยุธยา - ฮอลันดา
11,กนผ,3.0,13.3,16.3,Afternoon,2018-01-24,อยุธยา - อาเซียน
12,สอศ,8.0,9.0,16.3,All Day,NaT,หอประชุม


In [51]:
df_62 = pd.read_excel("../data/ห้องประชุม ไปราชการปี 2562.xlsx", header=0, skiprows=1)
print("DataFrame shape:", df_62.shape)

df_62 = preprocess_dataframe(df_62)
# Apply the function
df_62["duration_hours"], df_62["start_time"], df_62["end_time"] = zip(
    *df_62["เวลา"].apply(calculate_duration)
)

df_62["event_period"] = df_62.apply(
    lambda row: classify_event_simple(row["start_time"], row["end_time"]), axis=1
)

df_62["ใช้ในวันที่ (datetime)"] = df_62["ใช้ในวันที่"].apply(thai_to_datetime)

room_cols = [
        'ห้องประชุม สนอ.', 'ห้องประชุมตึก 70 ล้าน',
        'ห้องประชุม  1/2', 'อยุธยา - อาเซียน', 'อยุธยา - ฮอลันดา',
        'อยุธยา - โปรตุเกส', 'ห้องประชุม 317', 'หอประชุม'
]
df_62["room"] = df_62.apply(lambda row: extract_room(row, room_cols), axis=1)

columns_to_drop = [
        'เลขที่', 'วันที่', 'ชื่อ-สกุลผู้ขอใช้',
        'ใช้ในวันที่', 'เวลา', 'ห้องประชุม สนอ.', 'ห้องประชุมตึก 70 ล้าน',
        'ห้องประชุม  1/2', 'อยุธยา - อาเซียน', 'อยุธยา - ฮอลันดา',
        'อยุธยา - โปรตุเกส', 'ห้องประชุม 317', 'หอประชุม', 'เรื่อง',
]
df_62 = drop_unnecessary_columns(df_62, columns_to_drop)

# Save the cleaned DataFrame to a new Excel file
df_62.to_excel("../data/clean/cleaned_ห้องประชุม_2562.xlsx", index=False)

df_62.head(10)

DataFrame shape: (1205, 15)
Column names after stripping: ['เลขที่', 'หน่วยงานที่ขอใช้', 'วันที่', 'ชื่อ-สกุลผู้ขอใช้', 'ใช้ในวันที่', 'เวลา', 'ห้องประชุม สนอ.', 'ห้องประชุมตึก 70 ล้าน', 'ห้องประชุม  1/2', 'อยุธยา - อาเซียน', 'อยุธยา - ฮอลันดา', 'อยุธยา - โปรตุเกส', 'ห้องประชุม 317', 'หอประชุม', 'เรื่อง']


,หน่วยงานที่ขอใช้,duration_hours,start_time,end_time,event_period,ใช้ในวันที่ (datetime),room
1,สภา,3.0,13.3,16.3,Afternoon,2019-01-08,อยุธยา - อาเซียน
2,สภา,3.0,9.3,12.0,Morning,2019-01-09,อยุธยา - อาเซียน
3,ควท,3.0,13.0,15.3,Afternoon,2019-01-10,อยุธยา - โปรตุเกส
4,สภา,7.0,9.3,16.3,All Day,2019-01-11,อยุธยา - โปรตุเกส
5,สนอ,2.0,9.0,11.0,Morning,2019-01-11,อยุธยา - ฮอลันดา
6,สภา,6.0,9.3,15.0,All Day,2019-01-17,อยุธยา - อาเซียน
7,สภา,3.0,13.3,16.3,Afternoon,2019-01-23,อยุธยา - โปรตุเกส
8,สภา,7.0,9.3,16.3,All Day,2019-01-24,อยุธยา - โปรตุเกส
9,งานประกัน,4.0,13.0,16.3,Afternoon,2019-01-29,อยุธยา - อาเซียน
11,คมส,11.0,7.3,18.3,All Day,2019-01-09,ห้องประชุม 317


In [52]:
df_64 = pd.read_excel("../data/ห้องประชุม ไปราชการปี 2564.xlsx", header=0, skiprows=1)
print("DataFrame shape:", df_64.shape)

df_64 = preprocess_dataframe(df_64)
# Apply the function
df_64["duration_hours"], df_64["start_time"], df_64["end_time"] = zip(
    *df_64["เวลา"].apply(calculate_duration)
)

df_64["event_period"] = df_64.apply(
    lambda row: classify_event_simple(row["start_time"], row["end_time"]), axis=1
)

df_64["ใช้ในวันที่ (datetime)"] = df_64["ใช้ในวันที่"].apply(thai_to_datetime)

room_cols = [
        'สนามหญ้า อาคาร2', 'ห้องประชุมตึก 70 ล้าน',
        'ห้องประชุม  1/2', 'อยุธยา - อาเซียน', 'อยุธยา - ฮอลันดา',
        'อยุธยา - โปรตุเกส', 'ห้องประชุม 317', 'หอประชุม',
]
df_64["room"] = df_64.apply(lambda row: extract_room(row, room_cols), axis=1)

columns_to_drop = [
        'เลขที่', 'วันที่', 'ชื่อ-สกุลผู้ขอใช้',
        'ใช้ในวันที่', 'เวลา', 'สนามหญ้า อาคาร2', 'ห้องประชุมตึก 70 ล้าน',
        'ห้องประชุม  1/2', 'อยุธยา - อาเซียน', 'อยุธยา - ฮอลันดา',
        'อยุธยา - โปรตุเกส', 'ห้องประชุม 317', 'หอประชุม', 'เรื่อง',
]
df_64 = drop_unnecessary_columns(df_64, columns_to_drop)

# Save the cleaned DataFrame to a new Excel file
df_64.to_excel("../data/clean/cleaned_ห้องประชุม_2564.xlsx", index=False)

df_64.head(10)

DataFrame shape: (1205, 15)
Column names after stripping: ['เลขที่', 'หน่วยงานที่ขอใช้', 'วันที่', 'ชื่อ-สกุลผู้ขอใช้', 'ใช้ในวันที่', 'เวลา', 'สนามหญ้า อาคาร2', 'ห้องประชุมตึก 70 ล้าน', 'ห้องประชุม  1/2', 'อยุธยา - อาเซียน', 'อยุธยา - ฮอลันดา', 'อยุธยา - โปรตุเกส', 'ห้องประชุม 317', 'หอประชุม', 'เรื่อง']


,หน่วยงานที่ขอใช้,duration_hours,start_time,end_time,event_period,ใช้ในวันที่ (datetime),room
0,นิติการ,3.0,9.3,12.0,Morning,2021-01-12,อยุธยา - ฮอลันดา
1,สภา,5.0,9.3,14.0,All Day,2021-01-13,อยุธยา - อาเซียน
2,นิติการ,3.0,13.3,16.3,Afternoon,2021-01-13,อยุธยา - ฮอลันดา
3,นิติการ,3.0,9.3,12.0,Morning,2021-01-15,อยุธยา - ฮอลันดา
4,กนผ,3.0,13.3,16.3,Afternoon,2021-01-15,อยุธยา - อาเซียน
5,นิติการ,3.0,9.3,12.0,Morning,2021-01-27,อยุธยา-โปรตุเกส
6,นิติการ,3.0,9.3,12.0,Morning,2021-01-18,อยุธยา - ฮอลันดา
7,กบค,7.0,9.3,16.3,All Day,2021-01-18,อยุธยา - อาเซียน
8,นิติการ,3.0,9.3,12.0,Morning,2021-01-19,อยุธยา - ฮอลันดา
9,นิติการ,3.0,9.3,12.0,Morning,2021-01-22,อยุธยา - ฮอลันดา


In [53]:
df_67 = pd.read_excel("../data/ห้องประชุม ไปราชการปี 2567_2568.xlsx", header=0, skiprows=1, sheet_name=0)
print("DataFrame shape:", df_67.shape)

df_67 = preprocess_dataframe(df_67)
# Apply the function
df_67["duration_hours"], df_67["start_time"], df_67["end_time"] = zip(
    *df_67["เวลา"].apply(calculate_duration)
)

df_67["event_period"] = df_67.apply(
    lambda row: classify_event_simple(row["start_time"], row["end_time"]), axis=1
)

df_67["ใช้ในวันที่ (datetime)"] = df_67["ใช้ในวันที่"].apply(thai_to_datetime)

room_cols = [
        'สนามหญ้าอาคาร 2', 'บ้านพลูหลวง', 'ห้องประชุม 1',
        'อยุธยา - อาเซียน', 'อยุธยา - ฮอลันดา', 'อยุธยา - โปรตุเกส',
        'ห้องประชุม 317', 'หอประชุม'
]
df_67["room"] = df_67.apply(lambda row: extract_room(row, room_cols), axis=1)

columns_to_drop = [
    'เลขที่', 'วันที่', 'ชื่อ-สกุลผู้ขอใช้',
       'ใช้ในวันที่', 'เวลา', 'สนามหญ้าอาคาร 2', 'บ้านพลูหลวง', 'ห้องประชุม 1',
       'อยุธยา - อาเซียน', 'อยุธยา - ฮอลันดา', 'อยุธยา - โปรตุเกส',
       'ห้องประชุม 317', 'หอประชุม', 'เรื่อง'
]
df_67 = drop_unnecessary_columns(df_67, columns_to_drop)

# Save the cleaned DataFrame to a new Excel file
df_67.to_excel("../data/clean/cleaned_ห้องประชุม_2567.xlsx", index=False)

df_67.head(10)

DataFrame shape: (689, 15)
Column names after stripping: ['เลขที่', 'หน่วยงานที่ขอใช้', 'วันที่', 'ชื่อ-สกุลผู้ขอใช้', 'ใช้ในวันที่', 'เวลา', 'สนามหญ้าอาคาร 2', 'บ้านพลูหลวง', 'ห้องประชุม 1', 'อยุธยา - อาเซียน', 'อยุธยา - ฮอลันดา', 'อยุธยา - โปรตุเกส', 'ห้องประชุม 317', 'หอประชุม', 'เรื่อง']


,หน่วยงานที่ขอใช้,duration_hours,start_time,end_time,event_period,ใช้ในวันที่ (datetime),room
1,ประกัน,3.0,13.3,16.3,Afternoon,2024-01-03,อยุธยา - อาเซียน
2,กบค,7.0,9.3,16.3,All Day,2024-01-31,อยุธยา - อาเซียน
4,คคศ(อาจารย์),4.0,18.0,22.0,Night,2024-02-10,หอประชุม
5,กบศ,2.0,14.0,16.0,Afternoon,2024-01-04,อยุธยา -โปรตุเกส
6,กบศ,3.0,13.0,16.0,Afternoon,2024-01-05,อยุธยา - อาเซียน
7,นิติการ,3.0,9.3,12.0,Morning,2024-01-16,อยุธยา -โปรตุเกส
8,นิติการ,3.0,9.3,12.0,Morning,2024-01-16,อยุธยา - ฮอลันดา
9,นิติการ,3.0,9.3,12.0,Morning,2024-01-23,อยุธยา -โปรตุเกส
11,ประกัน,3.0,9.0,12.0,Morning,2024-01-02,หอประชุม
12,บัณฑิต,4.0,13.0,16.3,Afternoon,2024-01-21,317 บน


In [54]:
df_68 = pd.read_excel("../data/ห้องประชุม ไปราชการปี 2567_2568.xlsx", header=0, skiprows=1, sheet_name=1)
print("DataFrame shape:", df_68.shape)

df_68 = preprocess_dataframe(df_68)
# Apply the function
df_68["duration_hours"], df_68["start_time"], df_68["end_time"] = zip(
    *df_68["เวลา"].apply(calculate_duration)
)

df_68["event_period"] = df_68.apply(
    lambda row: classify_event_simple(row["start_time"], row["end_time"]), axis=1
)

df_68["ใช้ในวันที่ (datetime)"] = df_68["ใช้ในวันที่"].apply(thai_to_datetime)

room_cols = [
    'สนามหญ้าอาคาร 2', 'บ้านพลูหลวง', 'ห้องประชุม 1',
       'อยุธยา - อาเซียน', 'อยุธยา - ฮอลันดา', 'อยุธยา - โปรตุเกส',
       'ห้องประชุม 317', 'หอประชุม'
]
df_68["room"] = df_68.apply(lambda row: extract_room(row, room_cols), axis=1)

columns_to_drop = [
    'เลขที่', 'วันที่', 'ชื่อ-สกุลผู้ขอใช้',
       'ใช้ในวันที่', 'เวลา', 'สนามหญ้าอาคาร 2', 'บ้านพลูหลวง', 'ห้องประชุม 1',
       'อยุธยา - อาเซียน', 'อยุธยา - ฮอลันดา', 'อยุธยา - โปรตุเกส',
       'ห้องประชุม 317', 'หอประชุม', 'เรื่อง'
]
df_68 = drop_unnecessary_columns(df_68, columns_to_drop)

# Save the cleaned DataFrame to a new Excel file
df_68.to_excel("../data/clean/cleaned_ห้องประชุม_2568.xlsx", index=False)

df_68.head(10)

DataFrame shape: (104, 15)
Column names after stripping: ['เลขที่', 'หน่วยงานที่ขอใช้', 'วันที่', 'ชื่อ-สกุลผู้ขอใช้', 'ใช้ในวันที่', 'เวลา', 'สนามหญ้าอาคาร 2', 'บ้านพลูหลวง', 'ห้องประชุม 1', 'อยุธยา - อาเซียน', 'อยุธยา - ฮอลันดา', 'อยุธยา - โปรตุเกส', 'ห้องประชุม 317', 'หอประชุม', 'เรื่อง']


,หน่วยงานที่ขอใช้,duration_hours,start_time,end_time,event_period,ใช้ในวันที่ (datetime),room
1,อาคาร,2.0,10.0,12.0,Morning,2025-01-03,ห้องประชุม 1
2,นิติการ,3.0,13.3,16.3,Afternoon,2025-01-15,อยุธยา - ฮอลันดา
3,กบศ,4.0,9.0,13.0,Morning,2024-01-16,อยุธยา - อาเซียน
4,คคศ,6.0,16.3,22.0,All Day,2025-03-08,หอประชุม
6,สอศ,4.0,13.0,16.3,Afternoon,2025-01-10,อยุธยา - อาเซียน
7,ควท,2.0,11.0,13.0,Morning,2025-01-10,บ้านพลูหลวง
8,กบศ,3.0,9.0,12.0,Morning,2025-01-17,อยุธยา - อาเซียน
9,กบศ,4.0,13.0,16.3,Afternoon,2025-01-23,อยุธยา - โปรตุเกส
10,กบศ,3.0,9.0,12.0,Morning,2025-01-24,อยุธยา - อาเซียน
11,สภา,8.0,8.3,16.3,All Day,2025-01-08,อยุธยา - โปรตุเกส


In [130]:
# Concatenate all cleaned DataFrames
all_data = pd.concat([df_57, df_58, df_59, df_60_1, df_60_2, df_60_3, df_61, df_62, df_64, df_67, df_68], ignore_index=True)
all_data['room'] = all_data['room'].apply(normalize_room)
all_data = all_data.dropna()
print("All data shape after concatenation:", all_data.shape)
all_data.head()

All data shape after concatenation: (3679, 7)


,หน่วยงานที่ขอใช้,duration_hours,start_time,end_time,event_period,ใช้ในวันที่ (datetime),room
0,รพ.อยุธยา,4.0,18.0,22.0,Night,2013-09-27,หอประชุม
1,กบค,4.0,9.0,12.3,Morning,2013-10-29,อยุธยา-อาเซียน
3,กบค,2.0,13.3,15.3,Afternoon,2013-10-09,อยุธยา-ฮอลันดา
4,กบศ,3.0,9.3,12.3,Morning,2013-10-10,อยุธยา-ฮอลันดา
5,กบศ,8.0,8.0,16.0,All Day,2013-10-30,หอประชุม


In [131]:
all_data['room'].unique()

array(['หอประชุม', 'อยุธยา-อาเซียน', 'อยุธยา-ฮอลันดา', 'อยุธยา-โปรตุเกส',
       'ห้องประชุม 317', 'ห้องประชุม 1', 'ห้องประชุม 2',
       'ห้องประชุมบ้านพลูหลวง', 'ห้องประชุม สนอ.', 'ห้องประชุม 317 บน',
       'ห้องประชุม 317 ล่าง', 'สนามฟุตบอล'], dtype=object)

In [132]:
price_map = {
    'หอประชุม' : 2500,
    'อยุธยา-อาเซียน': 1000, 
    'อยุธยา-ฮอลันดา': 200, 
    'อยุธยา-โปรตุเกส': 250,
    'ห้องประชุม 317': 1000,  
    'ห้องประชุม 1': 400, 
    'ห้องประชุม 2': 650,
    'ห้องประชุมบ้านพลูหลวง': 650,
    'ห้องประชุม สนอ.': 2300, 
    'ห้องประชุม 317 บน': 1000,
    'ห้องประชุม 317 ล่าง': 1000,
    'สนามฟุตบอล': 3000,
}


seats_map = {
    'หอประชุม' : 700,
    'อยุธยา-อาเซียน': 30, 
    'อยุธยา-ฮอลันดา': 8, 
    'อยุธยา-โปรตุเกส': 12,
    'ห้องประชุม 317': 300,  
    'ห้องประชุม 1': 20, 
    'ห้องประชุม 2': 80,
    'ห้องประชุมบ้านพลูหลวง': 80,
    'ห้องประชุม สนอ.': 400, 
    'ห้องประชุม 317 บน': 180,
    'ห้องประชุม 317 ล่าง': 120,
    'สนามฟุตบอล': 1300,
}

In [133]:
all_data.columns

Index(['หน่วยงานที่ขอใช้', 'duration_hours', 'start_time', 'end_time',
       'event_period', 'ใช้ในวันที่ (datetime)', 'room'],
      dtype='object')

In [134]:
all_data['price'] = all_data['room'].map(price_map)
all_data['number_of_seats'] = all_data['room'].map(seats_map)

all_data = all_data.rename(columns={
    'หน่วยงานที่ขอใช้': 'department',
    'duration_hours': 'duration_hours',
    'start_time': 'start_time',
    'end_time': 'end_time',
    'event_period': 'event_period',
    'ใช้ในวันที่ (datetime)': 'date',
    'room': 'room',
    'price': 'price',
    'number_of_seats': 'seats'
})


all_data['date'] = all_data['date'].dt.date

all_data.head()

,department,duration_hours,start_time,end_time,event_period,date,room,price,seats
0,รพ.อยุธยา,4.0,18.0,22.0,Night,2013-09-27,หอประชุม,2500,700
1,กบค,4.0,9.0,12.3,Morning,2013-10-29,อยุธยา-อาเซียน,1000,30
3,กบค,2.0,13.3,15.3,Afternoon,2013-10-09,อยุธยา-ฮอลันดา,200,8
4,กบศ,3.0,9.3,12.3,Morning,2013-10-10,อยุธยา-ฮอลันดา,200,8
5,กบศ,8.0,8.0,16.0,All Day,2013-10-30,หอประชุม,2500,700


In [127]:
with pd.ExcelWriter(
    '../data/clean/cleaned_all_rooms.xlsx',
    engine='xlsxwriter',
    date_format='yyyy-mm-dd',            # บังคับให้ format ว่าเป็นวันที่
    datetime_format='yyyy-mm-dd'         # สำหรับ datetime ก็ใช้รูปแบบเดียวกัน
) as writer:
    all_data.to_excel(writer, index=False)